<a href="https://colab.research.google.com/github/Wnjoki/PILyTQOJRh9c4klH/blob/main/Potential_Talent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Libraries

In [2]:
# data processing
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

import re
import string
from collections import Counter

#Text-Processing
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


#models
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
#from sentence_transformers import SentenceTransformer
from transformers import BertTokenizer, BertModel
import torch






[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Load  and Explore Dataset

In [3]:
#load the data
talent_df = pd.read_csv("/content/potential-talents - Aspiring human resources - seeking human resources.csv")

In [4]:
talent_df.head()

,id,job_title,location,connection,fit
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN


In [5]:
talent_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          104 non-null    int64  
 1   job_title   104 non-null    object 
 2   location    104 non-null    object 
 3   connection  104 non-null    object 
 4   fit         0 non-null      float64
dtypes: float64(1), int64(1), object(3)
memory usage: 4.2+ KB


In [6]:
#Statistical analysis of categorical columns
talent_df.describe(include=object)

,job_title,location,connection
count,104,104,104
unique,52,41,33
top,2019 C.T. Bauer College of Business Graduate (...,Kanada,500+
freq,7,12,44


In [7]:
talent_df.shape

(104, 5)

In [8]:
len(talent_df['id'].unique())

104

In [9]:
print(talent_df.columns)

Index(['id', 'job_title', 'location', 'connection', 'fit'], dtype='object')


In [10]:
talent_df = talent_df.set_index('id')

talent_df.head()

,job_title,location,connection,fit
id,,,,
1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN
2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN
3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN


In [11]:
#drop fit column

talent_df.drop('fit', axis=1, inplace=True)

In [12]:
#count dublicated rows
talent_df.loc[talent_df.duplicated(), :]
print(talent_df.duplicated().sum())

51


In [13]:
#drop dublicated rows
talent_df.drop_duplicates(inplace=True)
talent_df.shape

(53, 3)

In [14]:
talent_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53 entries, 1 to 104
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   job_title   53 non-null     object
 1   location    53 non-null     object
 2   connection  53 non-null     object
dtypes: object(3)
memory usage: 1.7+ KB


In [15]:
#Verify if ther are some null values

talent_df.isnull().sum()

job_title     0
location      0
connection    0
dtype: int64

In [16]:
#value count of the job titles
talent_df.job_title.value_counts()

Aspiring Human Resources Professional                                                                                    2
2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources professional                 1
Lead Official at Western Illinois University                                                                             1
Senior Human Resources Business Partner at Heil Environmental                                                            1
Aspiring Human Resources Professional | An energetic and Team-Focused Leader                                             1
HR Manager at Endemol Shine North America                                                                                1
Human Resources professional for the world leader in GIS software                                                        1
RRP Brand Portfolio Executive at JTI (Japan Tobacco International)                                                       1
Information Syst

#Data Cleansing / Text Preprocessing

In [17]:
nlp = spacy.load("en_core_web_sm")

def pre_process(text, stem="None"):

    final_text = ""

    # Remove all the special characters
    text = re.sub(r'\W', ' ', str(text))

    # remove all single characters
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)

    # Remove single characters from the start
    text = re.sub(r'\^[a-zA-Z]\s+', ' ', text)

    # Removing prefixed 'b'
    text = re.sub(r'^b\s+', '', text)

    # Make text lower case
    text = text.lower()

    # Remove stop words
    text = text.split()
    Stopwords = set(stopwords.words('english'))

    text_filtered = [word for word in text if not word in Stopwords]

    # Remove numbers
    text_filtered = [re.sub(r'\w*\d\w*', '', w) for w in text_filtered]

    # Stem or Lemmatize
    if stem == 'Stem':
        stemmer = PorterStemmer()
        text_stemmed = [stemmer.stem(y) for y in text_filtered]
    elif stem == 'Lem':
        lem = WordNetLemmatizer()
        text_stemmed = [lem.lemmatize(y) for y in text_filtered]
    elif stem == 'Spacy':
        text_filtered = nlp(' '.join(text_filtered))
        text_stemmed = [y.lemma_ for y in text_filtered]
    else:
        text_stemmed = text_filtered

    final_text = ' '.join(text_stemmed)

    # Removing whitespace from the start of the string
    final_text = final_text.lstrip()

    # Substituting multiple spaces with single space
    final_text = re.sub(r'\s+', ' ', final_text, flags=re.I)

    return final_text

In [18]:
talent_df['job_title'] = talent_df['job_title'].apply(pre_process, stem='Spacy')

In [19]:
#Example after text preprocessing

#print(talent_df.iloc[0]['job_title'])
talent_df.iloc[0]['job_title']

'bauer college business graduate magna cum laude aspire human resource professional'

In [20]:
#Frequency distribution
plot_dist = talent_df['connection'] .value_counts().sort_index()
fig = px.bar(plot_dist,title='Frequency Distribution for all connections',text_auto=True)
fig.show()

500+ have the highest values

In [21]:
plot_dist = talent_df['location'] .value_counts().sort_index()
fig = px.bar(plot_dist,title='Frequency Distribution for all locations',text_auto=True)
fig.show()

Houston Texas Area has the highest number of candidates.

In [22]:
#value count of the job titles
talent_df.job_title.value_counts()

aspire human resource professional                                                                2
bauer college business graduate magna cum laude aspire human resource professional                1
lead official western illinois university                                                         1
senior human resource business partner heil environmental                                         1
aspire human resource professional energetic team focus leader                                    1
hr manager endemol shine north america                                                            1
human resource professional world leader gis software                                             1
rrp brand portfolio executive jti japan tobacco international                                     1
information system specialist programmer love datum organization                                  1
bachelor science biology victoria university wellington                                           1


Some of the job_title have some abbreviations to be expanded and also spacing to adjust.

In [23]:
#replace abreviations by there real texts
talent_df.replace({'job_title' : { 'chro' : 'chief human resources officer', 'svp' : 'senior vice president'
        ,'gphr' : 'global professional in human resources','hris' : 'human resources information system'
        , 'csr' : 'corporate social responsibility', 'sphr' : 'Senior Professional in Human Resources'
        , 'hr' : 'human resources', 'rrp': 'recommended retail price'}}, regex=True, inplace=True)

In [24]:
#word counts
words_counts = Counter()
for i in talent_df.job_title:
    for word in i.split(" "):
        words_counts[word] += 1

print('There are', len(words_counts) ,'words in the job title column')
words_counts.most_common()

There are 186 words in the job title column


[('human', 39),
 ('resource', 34),
 ('aspire', 13),
 ('professional', 11),
 ('seek', 10),
 ('manager', 7),
 ('university', 6),
 ('student', 6),
 ('management', 6),
 ('business', 5),
 ('generalist', 5),
 ('resources', 5),
 ('specialist', 4),
 ('position', 4),
 ('senior', 3),
 ('information', 3),
 ('system', 3),
 ('opportunity', 3),
 ('retail', 3),
 ('director', 3),
 ('major', 3),
 ('college', 2),
 ('english', 2),
 ('coordinator', 2),
 ('officer', 2),
 ('in', 2),
 ('internship', 2),
 ('staff', 2),
 ('north', 2),
 ('america', 2),
 ('leader', 2),
 ('bauer', 1),
 ('graduate', 1),
 ('magna', 1),
 ('cum', 1),
 ('laude', 1),
 ('native', 1),
 ('teacher', 1),
 ('epik', 1),
 ('program', 1),
 ('korea', 1),
 ('people', 1),
 ('development', 1),
 ('ryan', 1),
 ('advisory', 1),
 ('board', 1),
 ('member', 1),
 ('celal', 1),
 ('bayar', 1),
 ('humber', 1),
 ('chapman', 1),
 ('vice', 1),
 ('president', 1),
 ('chief', 1),
 ('marketing', 1),
 ('communication', 1),
 ('corporate', 1),
 ('social', 1),
 ('respo

Most frequent words ('human', 39),
 ('resource', 34),
 ('aspire', 13),
 ('professional', 11),
 ('seek', 10),

#Ranking/Score candidates based on keywords.

In [25]:
#create a df with relevant columns

df = talent_df[['job_title', 'connection']]
df.head()

,job_title,connection
id,,
1,bauer college business graduate magna cum laud...,85
2,native english teacher epik english program korea,500+
3,aspire human resource professional,44
4,people development coordinator ryan,500+
5,advisory board member celal bayar university,500+


#doc2vec embeddings

In [26]:
# Doc2Vec Embeddings
tagged_data = [TaggedDocument(words=title.split(), tags=[str(i)]) for i, title in enumerate(df['job_title'])]


In [27]:
# Train Doc2Vec model
model = Doc2Vec(tagged_data, vector_size=100, window=2, min_count=1, workers=4, epochs=20)


In [28]:
# Doc2Vec embedding for "Aspiring human resources" (w1)
w1 = "Aspiring human resources"
w1_vector = model.infer_vector(w1.split())


In [29]:
#  cosine similarity between w1 and job_title embeddings
cos_similarities= []
for i, title in enumerate(df['job_title']):
    title_words = title.split()
    similarity = cosine_similarity([model.infer_vector(title_words)], [w1_vector])[0][0]
    cos_similarities.append(similarity)

In [30]:
df['cos_similarities'] = cos_similarities

In [31]:
# Convert "connection" column to numeric format
df['connection'] = df.loc[:, 'connection'].str.replace('+', '').astype(float)


<ipython-input-31-acb48d79fc21>:2: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.



In [32]:
# Scale "connection" column between 0 and 1
scaler = MinMaxScaler()
df['scaled_connection'] = scaler.fit_transform(df[['connection']])


In [33]:
# Calculating ranking
df['ranking'] = 0.8 * df['cos_similarities'] + 0.2 * df['scaled_connection']


In [34]:
df = df.sort_values(by='ranking', ascending=False)
df

,job_title,connection,cos_similarities,scaled_connection,ranking
id,,,,,
71,human resource generalist scottmadden inc,500.0,0.253364,1.000000,0.402691
69,director human resource north america groupe b...,500.0,0.061954,1.000000,0.249563
101,human resource generalist loparex,500.0,0.057677,1.000000,0.246141
67,human resource staff recruit professional,500.0,0.037127,1.000000,0.229702
28,seek human resource opportunity,390.0,0.072950,0.779559,0.214272
81,senior human resource business partner heil en...,455.0,0.039400,0.909820,0.213484
13,human resource coordinator intercontinental bu...,500.0,0.016578,1.000000,0.213262
4,people development coordinator ryan,500.0,0.014368,1.000000,0.211494
5,advisory board member celal bayar university,500.0,0.013165,1.000000,0.210532


Bert Embeddings

In [35]:
# BERT Embeddings
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model_bert = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [36]:
#generate BERT embeddings
def get_bert_embedding(text):
    input_ids = torch.tensor(tokenizer.encode(text, add_special_tokens=True)).unsqueeze(0)
    outputs = model_bert(input_ids)
    embeddings = torch.mean(outputs.last_hidden_state, dim=1).squeeze(0).detach().numpy()
    return embeddings

In [37]:
# BERT embedding for "Aspiring human resources" (w1)
w1_embedding = get_bert_embedding(w1)

print(w1_embedding[:3])

[ 0.30581912 -0.08324541  0.24562995]


In [38]:
# Cosine similarity between w1 and job_title BERT embeddings
df['bert_embedding'] = df['job_title'].apply(get_bert_embedding)
df['cosine_similarity'] = df['bert_embedding'].apply(lambda x: cosine_similarity([x], [w1_embedding])[0][0])

In [39]:
# Scale "connection" column between 0 and 1
scaler = MinMaxScaler()
df['scaled_connection'] = scaler.fit_transform(df[['connection']])


In [40]:
# Ranking based on weighted sum of cosine_similarity and scaled_connection
df['ranking'] = 0.8 * df['cosine_similarity'] + 0.2 * df['scaled_connection']


In [41]:
# Sort the dataframe
df = df.sort_values(by='ranking', ascending=False)

df.head()

,job_title,connection,cos_similarities,scaled_connection,ranking,bert_embedding,cosine_similarity
id,,,,,,,
8,human resources senior specialist,500.0,-0.068909,1.0,0.828493,"[-0.15310206, -0.013271034, -0.02504923, -0.05...",0.785617
67,human resource staff recruit professional,500.0,0.037127,1.0,0.822245,"[0.20598105, 0.26471734, 0.0620079, 0.21260397...",0.777806
68,human resource specialist luxottica,500.0,-0.082961,1.0,0.806587,"[0.055227414, -0.11013625, 0.02690214, 0.26329...",0.758234
4,people development coordinator ryan,500.0,0.014368,1.0,0.805808,"[-0.22670178, -0.28873459, -0.3249328, 0.07421...",0.757261
104,director administration excellence log,500.0,-0.014782,1.0,0.805647,"[-0.22222485, -0.06335052, -0.27392873, 0.0145...",0.757059


In [42]:
#Top  candidates with highest ranking scores
top_candidates = df.head(10)

In [43]:
# Print the top candidates
top_candidates

,job_title,connection,cos_similarities,scaled_connection,ranking,bert_embedding,cosine_similarity
id,,,,,,,
8,human resources senior specialist,500.0,-0.068909,1.000000,0.828493,"[-0.15310206, -0.013271034, -0.02504923, -0.05...",0.785617
67,human resource staff recruit professional,500.0,0.037127,1.000000,0.822245,"[0.20598105, 0.26471734, 0.0620079, 0.21260397...",0.777806
68,human resource specialist luxottica,500.0,-0.082961,1.000000,0.806587,"[0.055227414, -0.11013625, 0.02690214, 0.26329...",0.758234
4,people development coordinator ryan,500.0,0.014368,1.000000,0.805808,"[-0.22670178, -0.28873459, -0.3249328, 0.07421...",0.757261
104,director administration excellence log,500.0,-0.014782,1.000000,0.805647,"[-0.22222485, -0.06335052, -0.27392873, 0.0145...",0.757059
75,nortia staff seek human resource payroll admin...,500.0,-0.071562,1.000000,0.795924,"[0.19250855, 0.25629637, 0.08381902, 0.3224198...",0.744905
27,aspire human resource management student seek ...,500.0,-0.010361,1.000000,0.784763,"[-0.10649364, 0.19814406, 0.28004533, 0.065952...",0.730953
89,director human resource ey,349.0,0.011595,0.697395,0.756666,"[-0.23601876, -0.058882218, -0.20357768, 0.164...",0.771484
101,human resource generalist loparex,500.0,0.057677,1.000000,0.756623,"[0.016921341, -0.19194953, 0.009764032, 0.2993...",0.695778


From the  above, BERT technique is more effective for ranking the talents and would be adopted. It's performance is because it's pretrained large corpus of unlabelled text.

#Re-ranking Candidates after starring

After starring a candidate(s) by entering the id, the job titles of that candidate(s) becomes the keyword and then the pointwise technique  would be used to re-rank the talent list to select the potential talents.

Re-rank using the job title starred as keyword and finding its cosine similarity to all the other job titles.
For this task Bert Embeddings vectors will be used.

In [44]:
index = df.index
print('Candidate Ids list')
for i in index:
    print(i,end=' ')

Candidate Ids list
8 67 68 4 104 75 27 89 101 71 94 10 78 28 81 13 2 77 103 12 97 69 3 99 85 74 82 66 6 76 73 84 83 72 7 92 102 79 100 80 88 5 98 70 1 86 87 90 96 11 91 95 93 

In [45]:
starred_id = input("Please, enter id of starred candidate(s): ")

Please, enter id of starred candidate(s): 27


In [47]:
# Build BERT_base model and get Bert vectors
from sentence_transformers import SentenceTransformer

bert_model = SentenceTransformer('bert-base-nli-mean-tokens')
rerank_key = df[df.index.isin([int(starred_id)])]['job_title'].tolist()
bert_vectors_rerank = bert_model.encode(list(df['job_title']))
rerank_vectors = bert_model.encode(rerank_key)

In [48]:
# Cosine similarity
RerankScore = []
for i in range(len(bert_vectors_rerank)) :
    re_rank_score = cosine_similarity(np.reshape(bert_vectors_rerank[i],(1,-1)), np.reshape(rerank_vectors[0],(1,-1)))
    RerankScore.append(re_rank_score[0][0])

df['Rerank Score'] = RerankScore

In [49]:
df.sort_values(by=['Rerank Score'],ascending=False)

,job_title,connection,cos_similarities,scaled_connection,ranking,bert_embedding,cosine_similarity,Rerank Score
id,,,,,,,,
27,aspire human resource management student seek ...,500.0,-0.010361,1.000000,0.784763,"[-0.10649364, 0.19814406, 0.28004533, 0.065952...",0.730953,1.000000
73,aspire human resource manager seek internship ...,7.0,-0.082749,0.012024,0.610330,"[-0.1271562, 0.17146829, 0.51870126, 0.0906924...",0.759906,0.889547
100,aspire human resource manager graduating may s...,103.0,-0.035948,0.204409,0.579319,"[-0.18393593, 0.18899688, 0.3001808, 0.0954751...",0.673046,0.714228
7,student humber college aspire human resource g...,61.0,0.041762,0.120240,0.593789,"[-0.08037113, 0.10052728, 0.31056604, 0.021812...",0.712176,0.710891
67,human resource staff recruit professional,500.0,0.037127,1.000000,0.822245,"[0.20598105, 0.26471734, 0.0620079, 0.21260397...",0.777806,0.707067
92,seek employment opportunity within customer se...,64.0,-0.012093,0.126253,0.590304,"[0.29570007, 0.23603885, 0.29177773, -0.117643...",0.706317,0.695936
90,undergraduate research assistant styczynski lab,155.0,-0.055698,0.308617,0.471541,"[0.15122992, -0.28775725, 0.1256062, -0.349010...",0.512272,0.684900
95,student westfield state university,57.0,0.128222,0.112224,0.431831,"[0.110543944, -0.37210092, -0.14186917, -0.144...",0.511733,0.626093
75,nortia staff seek human resource payroll admin...,500.0,-0.071562,1.000000,0.795924,"[0.19250855, 0.25629637, 0.08381902, 0.3224198...",0.744905,0.621064


The result above shows  the ranked list based on BERT's model being reranked successfully. The starred candidate with id 27 is labeled as 1. Thus our model achieves the goals of this project.

